# Dynamical Sampling on Graphs

In [515]:
###### nbi:hide_in

# Importing needed libraries 
import matplotlib
import networkx as nx
import random
import numpy as np
import copy
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

from scipy.sparse import csgraph
from scipy.sparse.linalg import eigsh, svds
from scipy.linalg import eigh

# Creating the graph class
class Graph(object):
    r"""
    Args:
        edges ([num_edges, 3] array): Graph connectivity in COO format 
        (instead of saving the adjacency matrix coo format saves only the node 
        values so the weights need to be given separetely). Third argument is 
        the weight. 
    """
    def __init__(self,  N_nodes=1, edges=[], samples=[], **kwargs):
        self.edges = edges
        self.N_nodes = N_nodes
        self.nodes = [i for i in range(N_nodes)]
        self.samples = samples
        
    def adj(self):
        adjacency_matr = np.zeros([self.N_nodes, self.N_nodes])
        for idx, row in enumerate(self.edges):
            ind1 = self.nodes.index(row[0])
            ind2 = self.nodes.index(row[1])
            adjacency_matr[ind1, ind2] = row[2]
            adjacency_matr[ind2, ind1] = adjacency_matr[ind1, ind2]
        return adjacency_matr
    
    def degrees(self):
        adj = self.adj()
        degrees = np.sum(adj, axis=0)
        return degrees
        
    def add_node(self):
        self.N_nodes += 1
        self.nodes.append(max(self.nodes)+1)
            
    def add_edge(self, edge):
        if edge!=None:
            self.edges.append(edge)
            
    def add_sample(self, node):
        if node not in self.samples:
            self.samples.append(node)
            
    def del_sample(self, node):
        if node in self.samples:
            self.samples.remove(node)
            
    def del_node(self, node):
        if node in self.nodes:
            self.N_nodes-=1
            self.edges = [item for item in self.edges if item[0]!=node and item[1]!=node]
            self.nodes.remove(node)
            self.del_sample(node)
    
    def del_edge(self, pair):
        self.edges[:] = [item for item in self.edges if item[:2]!=pair and item[:2]!=(pair[1], pair[0])]
        
    def change_edge(self, newedge):
        for edge in self.edges:
            if (edge[0], edge[1])==(newedge[0], newedge[1]) or (edge[1], edge[0])==(newedge[0], newedge[1]):
                self.del_edge((newedge[0], newedge[1]))
                self.add_edge(newedge)
                       
    #reset graph
    def reset(self):
        self.N_nodes = 1
        self.nodes = [i for i in range(self.N_nodes)]
        self.edges = []
    
    def lapl_eigen(self, dim=None):
        Adj = self.adj()
        Lap = csgraph.laplacian(Adj, normed=False)
        if dim==None:
            dim=G.N_nodes
        vals, U = eigh(Lap, subset_by_index=[0,dim-1])
        return vals, U
    
    def adjacent2(self):
        """Return the adjoint nodes for given node"""
        adjacency = {node:[] for node in self.nodes}
        for edge in self.edges:
            adjacency[edge[0]].append(edge[1])
            adjacency[edge[1]].append(edge[0])
        return adjacency
    
    def is_connected(self):
        """Check if the graph is connected using width-first search"""
        adjacency = self.adjacent2()
        count=0
        found = {i:False for i in self.nodes}
        Q = []
        Q.append(0)
        while Q: # checks if Q is empty
            nhbs = adjacency[Q[0]]
            for node in nhbs:
                if found[node]==False:
                    count+=1
                    found[node]=True
                    Q.append(node)
            Q.pop(0)
        if count==self.N_nodes:
            return True
        else:
            return False

In [558]:
###### nbi:hide_in
%matplotlib inline

from IPython.display import display, clear_output
import ipywidgets as widgets
from ipywidgets import Button, Layout, GridspecLayout

def draw_graph(G, ax, output, k=None, labels=None): 
    #create the graph
    Gnx = nx.Graph()
    Gnx.add_nodes_from(G.nodes)
    Gnx.add_weighted_edges_from(G.edges)
    
    # colors
    if k!=None:
        colors = plt.cm.get_cmap('tab20', k)
        color_label = colors(labels)
        node_colors = [(0.0, 0.0, 0.0, 1.0) if node in G.samples else color_label[node] for node in G.nodes]
    else:
        node_colors = ["black" if node in G.samples else "blue" for node in G.nodes]
        
    #plot
    with output:
        ax.cla()
        nx.draw_networkx(Gnx, ax=ax, node_color=node_colors)
        display(fig); 

def dynamic(A, L, V):
    Mat = np.eye(A.shape[0])
    for i in range(L-1):
        Mat = np.concatenate([np.eye(A.shape[0]), Mat @ A])
    F = Mat @ V
    return F.reshape(A.shape[0], L*V.shape[1], order="F")
            
def gds(G, pw_dim, L, output, options=0):
    # sampling matrix
    S = np.zeros([G.N_nodes, len(G.samples)])
    for j, node in enumerate(G.samples):
        i = G.nodes.index(node)
        S[i, j]=1
        
    Adj = G.adj() # the adjoint matrix
    Lap = csgraph.laplacian(Adj, normed=False)
    
    # Compute PW eigenvectors
    vals, U =  eigh(Lap, subset_by_index=[0,pw_dim-1])
                            
    # Compute the dynamical sampling vectors
    if options==0:
        B = dynamic(Lap, L, S)
    if options==1:
        B = dynamic(Adj, L, S)
                            
    # Project onto PW space
    PF = U.transpose() @ B
                            
    # Compute frame bounds
    Frame_op = PF @ PF.transpose()
    low = svds(Frame_op, k=1, which='SM', return_singular_vectors=False)[0]
    up = svds(Frame_op, k=1, which='LM', return_singular_vectors=False)[0]

    # print
    with output:
        display("Lower frame bound = {}".format(low))
        display("Upper frame bound = {}".format(up))

In [559]:
# The figure
fig, ax = plt.subplots(figsize=(10, 5))
ax.spines["top"].set_visible(False)  
ax.spines["right"].set_visible(False)
ax.spines["bottom"].set_visible(False)
ax.spines["left"].set_visible(False)
plt.close()

output = widgets.Output()


# Generate a random connected graph           
def random_connected_graph(N_nodes, N_edges):
    """Uses rejection sampling to uniformly sample a connected graph"""
    G = Graph(N_nodes)
    
    if N_edges<N_nodes-1:
        raise ValueError("Not enough edges")
    if N_edges>N_nodes*(N_nodes):
        raise ValueError("Too many edges")
    all_edges = [(i,j,1) for i in range(N_nodes) for j in range(i)]
    while True:
        G.edges = random.sample(all_edges, N_edges)
        if G.is_connected():
            break
        # breadth first search to determine if it is connected
    return G

In [560]:
# Samples from the cluster
def Kmeans(X, k, N_iter=30):
    """Assigns the rows of X into k clusters"""
    dim = X.shape[1]
    C = np.random.rand(k, dim) # initialize
    
    def pairwise_dist(X, C):
        """returns array of size [dim, k]"""
        distances = np.linalg.norm(X[:,None,:]-C[None,:,:], axis=-1)
        return distances
    
    def get_clusters(C):
        clusters = {i:[] for i in range(k)}
        distances = pairwise_dist(X, C)
        labels = list(np.argmin(distances, axis=1))
        for i in range(X.shape[0]):
            clusters[labels[i]].append(i)
        return clusters, labels

    def means(clusters):
        C_new = np.zeros([k, dim])
        for i in range(k):
            if clusters[i]:
                C_new[i,:] = np.mean(X[clusters[i],:], axis=0)
            else:
                C_new[i,:] = np.random.randn(1,dim)
        return C_new
    
    for i in range(N_iter):
        clusters, labels = get_clusters(C)
        C = means(clusters)
        
    return labels, clusters

def clustered_samples(clusters):
    """In each cluster pick the node with largest degree"""
    degrees = G.degrees()
    samples = [clusters[key][np.argmax(degrees[clusters[key]])] for key in clusters.keys()]
    return samples  

In [561]:
# Samples from the cluster using sklearn

def Kmeans_sklrn(X, k):
    """Assigns the rows of X into k clusters"""
    kmeans = KMeans(n_clusters=k).fit(X)
    labels = list(kmeans.labels_)
    
    clusters = {i:[] for i in range(k)}
    
    for indx, item in enumerate(labels):
        clusters[item].append(indx)
    
    return labels, clusters

In [569]:
#### N_nodes=30
N_edges=34
N_samples=2
pw_dim=5
L=6

# Generate a random graph
G = random_connected_graph(N_nodes, N_edges)

# add eigenvalues to the output
output.clear_output()
eig, U = G.lapl_eigen()
with output:
    display("Laplacian eigenvalues are {}".format(eig))

In [570]:

_, X = G.lapl_eigen(N_samples)

# graph sampled with spectral clusters
labels, clusters = Kmeans_sklrn(X, k=N_samples) 
G.samples = clustered_samples(clusters)
gds(G, pw_dim, L, output)
draw_graph(G, ax, output, N_samples, labels)

# # graph sampled with spectral clusters with sklrn
# G2=copy.deepcopy(G)
# labels2, clusters2 = Kmeans_sklrn(X, k=N_samples) 
# G2.samples = clustered_samples(clusters2)
# gds(G2, pw_dim, L, output)
# draw_graph(G2, ax, output, N_samples, labels2)

# graph with sampled randomly
G1=copy.deepcopy(G)
G1.samples = random.sample(G1.nodes, N_samples)
gds(G1, pw_dim, L, output)
draw_graph(G1, ax, output)

display(output)   



Output(outputs=({'output_type': 'display_data', 'data': {'text/plain': "'Laplacian eigenvalues are [6.26441964…